In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from database.models import *  # Importiert z. B. auch die Klasse Konzept

In [2]:
# Verbindung zur Datenbank herstellen
DATABASE_URL = "sqlite:///database.db"  # Pfad zur SQLite-Datenbank
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

In [3]:
# Einen Inspector erstellen, um Metadaten der DB abzufragen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
tabellen = inspector.get_table_names()

# Ausgabe der gefundenen Tabellen
print("Gefundene Tabellen in der Datenbank:")
for tabelle in tabellen:
    print(tabelle)

Gefundene Tabellen in der Datenbank:
DPM_data_points
DPM_tableStructure
Finrep_Y_reference
ITSBaseData_new
ITS_base_data
ITS_base_data_new
Konzepte
MergedData
Template_Finrep


## FINREP Y Reference

In [6]:
def get_entries_by_code():
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN0000001'
        entries = session.query(Finrep_Y_reference).all()
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
df2 = get_entries_by_code()
df2.head()

""


In [7]:
def get_first_5_entries_df():
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM konzept ORDER BY id LIMIT 5
        entries = session.query(MergedData).order_by(MergedData.id).all()
        #entries = session.query(Konzept).order_by(Konzept.id).all()
        data = []
        for entry in entries:
            # Konvertiere jeden Datensatz in ein Dictionary
            # Entferne dabei den internen SQLAlchemy State (_sa_instance_state)
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        # Erstelle ein DataFrame aus der Liste der Dictionaries
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

df = get_first_5_entries_df()
df.head(1)

,template_id,x_axis_rc_code,member_name,template_label,x_axis_name,konzept_id,module_id,y_axis_rc_code,id,module_gueltig_von,y_axis_name,its_base_data_id,module_gueltig_bis,z_axis_rc_code,datapoint,table_id,z_axis_name,konzept_code,table_name,datapoint_vid,taxonomy_code,criteria,dimension_label
0,F 14.00,0060,Financial assets held for trading,Fair value hierarchy: financial instruments at...,Level 1,93221,FINREP9,0040,13826,2022-12-31,Debt securities,1,3099-12-31 00:00:00,None,11133,F 14.00,None,ISFIN0001590,Fair value hierarchy: financial instruments at...,11133,FINREP_3.2.1,None,Accounting portfolio


### Konzept

In [8]:
def get_entries_by_code(code_value="ISFIN0000001"):
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN2022000'
        entries = session.query(Konzept).filter(Konzept.code == code_value).order_by(Konzept.id).all()
        #entries = session.query(Konzept).all()
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
df1 = get_entries_by_code()
df1 

,id,scs_einschraenkung,gueltig_von,smart_cube_konzept,anubis_rechenregel,gueltig_bis,pflichtkonzept,aggregationstyp,mdi_relevant,dimensionskombination,kurzbezeichnung,mdi_modellierungstyp,dimensionen,kurzbezeichnung_englisch,erhebungsteile,konzepttyp,bezeichnung,observ_schluesselgruppe,bezeichnung_englisch,code,beschreibung
0,91698,None,2014-09-30,Nein,None,None,True,Bestandsaggregation,True,OHNE (Ohne Dimension),"KassBest u Guth ZB, Kas",Template,None,Cash and cash bal at CB,"51-F01.01, 56-F01.01, FINAI-0101, FINAU-0101, ...",Wert,"KassBest u Guth ZB, KassBest, BW",None,"Cash, cash balances at central banks and other...",ISFIN0000001,None


### ITS Base Data

In [6]:
from sqlalchemy import and_

def get_entries_by_code(code_value="ISFIN0009969"):
    session = SessionLocal()
    try:
        entries = session.query(ITSBaseData_new).filter(
            and_(
                #ITSBaseData_new.konzept_code == code_value
                ITSBaseData_new.taxonomy_code == 'COREP_3.2'
                #ITSBaseData_new.y_axis_rc_code == '0070'  # <--- hier die zusätzliche Spalte
            )).all()
        data = []
        for entry in entries: 
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally: session.close()
pd.set_option('display.max_columns', None)
df2 = get_entries_by_code()
df2.head(100)

#df2['module_id'].unique()

,datapoint,ko,template_code,module_code,module_gueltig_bis,table_name,x_axis_rc_code,y_axis_rc_code,z_axis_rc_code,taxonomy_code,id,template_label,module_gueltig_von,table_code,criteria,x_axis_name,y_axis_name,z_axis_name
0,NaN,None,C 74.00,COREP_LCR_DA,3099-12-31 00:00:00,Liquidity Coverage. Inflows. Significant curre...,0100,0439,999,COREP_3.2,5,(LC(DA) - Inflows) Liquidity Coverage. Inflows...,2022-12-31,C 74.00.w,None,Exempted from the cap on inflows,MEMORANDUM ITEMS,Significant currency
1,NaN,None,C 74.00,COREP_LCR_DA,3099-12-31 00:00:00,Liquidity Coverage. Inflows. Significant curre...,0020,0439,999,COREP_3.2,6,(LC(DA) - Inflows) Liquidity Coverage. Inflows...,2022-12-31,C 74.00.w,None,Subject to the 90% cap on inflows,MEMORANDUM ITEMS,Significant currency
2,NaN,None,C 74.00,COREP_LCR_DA,3099-12-31 00:00:00,Liquidity Coverage. Inflows. Significant curre...,0060,0529,999,COREP_3.2,7,(LC(DA) - Inflows) Liquidity Coverage. Inflows...,2022-12-31,C 74.00.w,None,Exempted from the cap on inflows,Secured lending waived from Article 17 (2) and...,Significant currency
3,NaN,None,C 74.00,COREP_LCR_DA,3099-12-31 00:00:00,Liquidity Coverage. Inflows. Significant curre...,0080,0439,999,COREP_3.2,8,(LC(DA) - Inflows) Liquidity Coverage. Inflows...,2022-12-31,C 74.00.w,None,Subject to the 75% cap on inflows,MEMORANDUM ITEMS,Significant currency
4,NaN,None,C 74.00,COREP_LCR_DA,3099-12-31 00:00:00,Liquidity Coverage. Inflows. Significant curre...,0120,0439,999,COREP_3.2,9,(LC(DA) - Inflows) Liquidity Coverage. Inflows...,2022-12-31,C 74.00.w,None,Subject to the 90% cap on inflows,MEMORANDUM ITEMS,Significant currency
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,None,C 66.01,COREP_ALM,3099-12-31 00:00:00,Maturity ladder. Total. Overnight and higher m...,0090,1089,0010,COREP_3.2,100,Maturity Ladder,2023-06-30,C 66.01.a,None,Greater than 7 days up to 2 weeks,CONTINGENCIES,Total currencies
96,NaN,None,C 66.01,COREP_ALM,3099-12-31 00:00:00,Maturity ladder. Total. Overnight and higher m...,0190,0389,0010,COREP_3.2,101,Maturity Ladder,2023-06-30,C 66.01.a,None,Greater than 9 months up to 12 months,INFLOWS,Total currencies
97,NaN,None,C 66.01,COREP_ALM,3099-12-31 00:00:00,Maturity ladder. Total. Overnight and higher m...,0160,1149,0010,COREP_3.2,102,Maturity Ladder,2023-06-30,C 66.01.a,None,Greater than 4 months up to 5 months,MEMORANDUM ITEMS,Total currencies
98,NaN,None,C 66.01,COREP_ALM,3099-12-31 00:00:00,Maturity ladder. Total. Overnight and higher m...,0040,0389,0010,COREP_3.2,103,Maturity Ladder,2023-06-30,C 66.01.a,None,Greater than 2 days up to 3 days,INFLOWS,Total currencies


### DPM

In [9]:
def get_entries_by_code(code_value="151111"):
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN0000001'
        entries = session.query(DPM_datapoint).filter(DPM_datapoint.datapoint_vid == code_value).order_by(DPM_datapoint.id).all()
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
df2 = get_entries_by_code()
df2
#df2.to_csv('ISFIN0000001.csv')

,dimension_label,id,member_name,datapoint_id,datapoint_vid
0,Accounting portfolio,618517,Cash and cash balances at central banks and ot...,32237,151111
1,Metric,618518,Carrying amount [mi],32237,151111
2,Base,618519,Assets,32237,151111
3,Counterparty sector,618520,Central banks,32237,151111
4,Instrument,618521,On demand [call] and short notice [current acc...,32237,151111
5,Main category,618522,Loans and advances,32237,151111


### Merged Data

In [21]:
def get_entries_by_code(code_value="ISFIN0000001"):
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN0000001'
        entries = session.query(MergedData).filter(MergedData.konzept_code == code_value).order_by(MergedData.id).all()
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
df2 = get_entries_by_code()
df2

,konzept_code,table_name,z_axis_name,taxonomy_code,criteria,datapoint_vid,template_id,x_axis_rc_code,dimension_label,template_label,x_axis_name,member_name,konzept_id,module_id,y_axis_rc_code,id,module_gueltig_von,y_axis_name,its_base_data_id,module_gueltig_bis,z_axis_rc_code,datapoint,table_id
0,ISFIN0000001,Balance Sheet Statement [Statement of Financia...,None,FINREP_3.2.1,None,112718,F 01.01,0010,Accounting portfolio,Balance Sheet Statement [Statement of Financia...,Carrying amount,Cash and cash balances at central banks and ot...,91698,FINREP9,0020,52880,2022-12-31,Cash on hand,5973,3099-12-31 00:00:00,None,112718,F 01.01
1,ISFIN0000001,Balance Sheet Statement [Statement of Financia...,None,FINREP_3.2.1,None,112718,F 01.01,0010,Base,Balance Sheet Statement [Statement of Financia...,Carrying amount,Assets,91698,FINREP9,0020,52881,2022-12-31,Cash on hand,5973,3099-12-31 00:00:00,None,112718,F 01.01
2,ISFIN0000001,Balance Sheet Statement [Statement of Financia...,None,FINREP_3.2.1,None,112718,F 01.01,0010,Main category,Balance Sheet Statement [Statement of Financia...,Carrying amount,Cash on hand,91698,FINREP9,0020,52882,2022-12-31,Cash on hand,5973,3099-12-31 00:00:00,None,112718,F 01.01
3,ISFIN0000001,Balance Sheet Statement [Statement of Financia...,None,FINREP_3.2.1,None,112718,F 01.01,0010,Metric,Balance Sheet Statement [Statement of Financia...,Carrying amount,Carrying amount [mi],91698,FINREP9,0020,52883,2022-12-31,Cash on hand,5973,3099-12-31 00:00:00,None,112718,F 01.01


In [22]:
df3 = df2[['konzept_code', 'dimension_label', 'member_name']]

df3.to_csv('meine_datei.csv', index=False)


## Template

In [8]:
def get_entries_by_code():
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN0000001'
        #entries = session.query(Template_Finrep).filter(Template_Finrep.datapoint_vid == code_value).all()
        
        entries = session.query(Template_Finrep)
        
        #entries = session.query(Template_Finrep).filter(
        #Template_Finrep.axis == 'y').all()
        #Template_Finrep.table == '1.1 Assets').all()
        #Template_Finrep.text == 'Cash on hand').all()
        
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df2 = get_entries_by_code()
#df2.drop(df2.columns[-1], axis=1).to_csv('refs.csv', index=False)

In [5]:
pd.set_option('display.max_rows', None)
df2.head()

,table,coord,template_sheet,reference,axis,id,text,extra
0,1.1 Assets,0010,1,Annex V.Part 1.27,x,1,Carrying amount,None
1,1.2 Liabilities,0010,1,Annex V.Part 1.27,x,2,Carrying amount,None
2,1.3 Equity,0010,1,Annex V.Part 1.27,x,3,Carrying amount,None
3,4.1 Financial assets held for trading,0010,4,Annex V.Part 1.27,x,4,Carrying amount,None
4,4.2.1 Non-trading financial assets mandatorily...,0010,4,Annex V.Part 1.27,x,5,Carrying amount,None


In [12]:
filtered_df = df2[df2['text'].str.contains('Cash and cash balances', na=False)]
filtered_df

,table,coord,template_sheet,reference,axis,id,text,extra


## Table Strucutre Hierarchy 

In [7]:
def get_entries_by_code():
    
    session = SessionLocal()
    try:
        # Abfrage: SELECT * FROM Konzept WHERE code = 'ISFIN0000001'
        entries = session.query(DPM_tableStructure).filter(DPM_tableStructure.componentcode == '20').all()
        
        #entries = session.query(DPM_tableStructure)
        
        #entries = session.query(Template_Finrep).filter(
        #Template_Finrep.axis == 'y').all()
        #Template_Finrep.table == '1.1 Assets').all()
        #Template_Finrep.text == 'Cash on hand').all()
        
        data = []
        for entry in entries:
            row = entry.__dict__
            row.pop("_sa_instance_state", None)
            data.append(row)
        df = pd.DataFrame(data)
        return df
    finally:
        session.close()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df2 = get_entries_by_code()
df2.head()
#df2.drop(df2.columns[-1], axis=1).to_csv('refs.csv', index=False)

,componentcode,id,headerflag,order,displaybeforechildren,parentordinateide,componentlabel,tablecode,componenttypename,level,taxonomycode,ordinateid
0,20,3,False,1020,1,70668,Cash on hand,F 01.01 - Balance Sheet Statement [Statement o...,Table row,3,FINREP 3.2.1,70669
1,20,65,False,1020,1,70718,Derivatives,F 01.02 - Balance Sheet Statement [Statement o...,Table row,3,FINREP 3.2.1,70719
2,20,108,False,1020,1,70761,Paid up capital,F 01.03 - Balance Sheet Statement [Statement o...,Table row,3,FINREP 3.2.1,70762


In [7]:
import pickle

# Lade die Pickle-Datei
with open("baumstruktur.pkl", "rb") as f:
    data = pickle.load(f)

# Überprüfe, ob es sich um eine Liste handelt
if isinstance(data, list):
    for idx, item in enumerate(data):
        print(f"Element {idx}: {item}")
else:
    print("Die geladene Struktur ist keine Liste.")


Element 0: Node(70715.0, Total assets)


### Löschen

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from database.models import *  # Stelle sicher, dass das Modell 'Konzepte' in database/models.py definiert ist


# Verbindung zur Datenbank herstellen
DATABASE_URL = "sqlite:///database.db"  # Pfad zur SQLite-Datenbank
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

def clear_konzepte_table():
    session = SessionLocal()
    try:
        # Löscht alle Zeilen aus der Tabelle Konzepte
        session.query(Finrep_Y_reference).delete()
        session.commit()
        print("Alle Einträge in der Tabelle 'Finrep_Y_reference' wurden gelöscht.")
    except Exception as e:
        session.rollback()
        print("Fehler beim Löschen der Einträge:", e)
    finally:
        session.close()

if __name__ == "__main__":
    clear_konzepte_table()

Alle Einträge in der Tabelle 'Finrep_Y_reference' wurden gelöscht.


## Drop Table

In [2]:
DATABASE_URL = "sqlite:///database.db"  # Pfad zur SQLite-Datenbank
engine = create_engine(DATABASE_URL)
Finrep_Y_reference.__table__.drop(engine)
print("complete")

complete
